In [3]:
!pip install paddle2onnx onnxruntime onnxslim onnxruntime-tools onnx pickleshare -i https://pypi.tuna.tsinghua.edu.cn/simple

In [1]:
!paddle2onnx --model_dir ../rapid_table_det_paddle/models --model_filename obj_det_paddle.pdmodel --params_filename obj_det_paddle.pdiparams --save_file ../rapid_table_det/models/obj_det.onnx --opset_version 16 --enable_onnx_checker True
!paddle2onnx --model_dir ../rapid_table_det_paddle/models --model_filename edge_det_paddle.pdmodel --params_filename edge_det_paddle.pdiparams --save_file ../rapid_table_det/models/edge_det.onnx --opset_version 16 --enable_onnx_checker True
!paddle2onnx --model_dir ../rapid_table_det_paddle/models --model_filename cls_det_paddle.pdmodel --params_filename cls_det_paddle.pdiparams --save_file ../rapid_table_det/models/cls_det.onnx --opset_version 16 --enable_onnx_checker True

!onnxslim ../rapid_table_det/models/obj_det.onnx ../rapid_table_det/models/obj_det.onnx
!onnxslim ../rapid_table_det/models/edge_det.onnx ../rapid_table_det/models/edge_det.onnx
!onnxslim ../rapid_table_det/models/cls_det.onnx ../rapid_table_det/models/cls_det.onnx

[Paddle2ONNX] Start to parse PaddlePaddle model...
[Paddle2ONNX] Model file path: ../rapid_table_det_paddle/models\obj_det_paddle.pdmodel
[Paddle2ONNX] Paramters file path: ../rapid_table_det_paddle/models\obj_det_paddle.pdiparams
[Paddle2ONNX] Start to parsing Paddle model...
[Paddle2ONNX] Use opset_version = 16 for ONNX export.
[WARN][Paddle2ONNX] [multiclass_nms3: multiclass_nms3_0.tmp_1] [WARNING] Due to the operator multiclass_nms3, the exported ONNX model will only supports inference with input batch_size == 1.
[Paddle2ONNX] PaddlePaddle model is exported as ONNX format now.
2024-10-19 21:58:34 [INFO]	===============Make PaddlePaddle Better!================
2024-10-19 21:58:34 [INFO]	A little survey: https://iwenjuan.baidu.com/?code=r8hu2s
[Paddle2ONNX] Start to parse PaddlePaddle model...
[Paddle2ONNX] Model file path: ../rapid_table_det_paddle/models\edge_det_paddle.pdmodel
[Paddle2ONNX] Paramters file path: ../rapid_table_det_paddle/models\edge_det_paddle.pdiparams
[Paddle2ONN

Onnx Runtime version 1.19 has no specified compatible ONNX version.


+-------------------------+----------------------------------------+----------------------------------------+
|       Model Name        |             edge_det.onnx              |               Op Set: 16               |
+-------------------------+----------------------------------------+----------------------------------------+
|       Model Info        |             Original Model             |             Slimmed Model              |
+-------------------------+----------------------------------------+----------------------------------------+
|        IN: input        | float32: ('p2o.DynamicDimension.0', 3, | float32: ('p2o.DynamicDimension.0', 3, |
|                         |               800, 800)                |               800, 800)                |
|        IN: input        | float32: ('p2o.DynamicDimension.0', 3, | float32: ('p2o.DynamicDimension.0', 3, |
|                         |               800, 800)                |               800, 800)                |
| OUT: rec

Onnx Runtime version 1.19 has no specified compatible ONNX version.


+---------------------+----------------------------------------+----------------------------------------+
|     Model Name      |              cls_det.onnx              |               Op Set: 16               |
+---------------------+----------------------------------------+----------------------------------------+
|     Model Info      |             Original Model             |             Slimmed Model              |
+---------------------+----------------------------------------+----------------------------------------+
|      IN: input      | float32: ('p2o.DynamicDimension.0', 3, | float32: ('p2o.DynamicDimension.0', 3, |
|                     |               624, 624)                |               624, 624)                |
|      IN: input      | float32: ('p2o.DynamicDimension.0', 3, | float32: ('p2o.DynamicDimension.0', 3, |
|                     |               624, 624)                |               624, 624)                |
| OUT: linear_1.tmp_1 | float32: ('p2o.Dynamic

Onnx Runtime version 1.19 has no specified compatible ONNX version.


In [2]:
from pathlib import Path
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType, quant_pre_process
def quantize_model(root_dir_str, model_dir, pre_fix):

    original_model_path = f"{pre_fix}.onnx"
    quantized_model_path = f"{pre_fix}_quantized.onnx"
    # quantized_model_path = original_model_path
    original_model_path = f"{root_dir_str}/{model_dir}/{original_model_path}"
    quantized_model_path = f"{root_dir_str}/{model_dir}/{quantized_model_path}"
    quant_pre_process(original_model_path, quantized_model_path, auto_merge=True)
    # 进行动态量化
    quantize_dynamic(
        model_input=quantized_model_path,
        model_output=quantized_model_path,
        weight_type=QuantType.QUInt8
    )

    # 验证量化后的模型
    quantized_model = onnx.load(quantized_model_path)
    onnx.checker.check_model(quantized_model)
    print("Quantized model is valid.")

In [3]:
root_dir_str = ".."
model_dir = f"rapid_table_det/models"
quantize_model(root_dir_str, model_dir,  "obj_det")
quantize_model(root_dir_str, model_dir,  "edge_det")

Quantized model is valid.
Quantized model is valid.


In [4]:
from fix_onnx import fix_onnx
import os
# 指定目录路径
model_dir = "../rapid_table_det/models"
# 加载现有 ONNX 模型
model_path = os.path.join(model_dir, "obj_det.onnx")
fix_onnx(model_path)
model_path = os.path.join(model_dir, "obj_det_quantized.onnx")
fix_onnx(model_path)